# Xerrada Ansible - Python Girona

----

Jaume Flórez Valenzuela
(amb la colaboració de Manel Clos)

# Què és Ansible

- Ansible és una eina que ens permet instalar i actualitzar paquets remotament.
- Ansible és una aplicació server-side, és a dir, que no l'hem d'instalar a tots els clients.

## Com l'Instalem?

Ansible és un paquet de python, així que com amb la resta de paquets utilitzarem pip

`pip install ansible`

Ansible també permet instalar sense utilitzar pip, però s'utilitza altre software.
Ho podeu veure revisant la [documentació](https://docs.ansible.com/ansible/2.4/intro_installation.html)

# Idempotència

Una vegada instalat tot el playbook, s'hauria de poder executar de nou sense realitzar cap canvi.
Això ens permet actualitzacions/instalacions massives amb diferents entorns.

Es basa amb el terme matemàtic on identifiquem que una operació és indempotent quan
en aplicarla a un terme, el resultat és el mateix que el terme.

>“_En matemática y lógica, la idempotencia es la propiedad para realizar una acción determinada varias veces y aun así conseguir el mismo resultado que se obtendría si se realizase una sola vez_”

# Tipus de Fitxers

- **Inventaris**: Contenen informació sobre els diferents hosts on aplicar les tasques.
- **Variables**: Fitxers on podem definir un seguit de variables per utilitzar en les tasques.
- **Playbook**: Fitxer on declarem les diferents tasques a realitzar.

# Estructura recomanada (I)

>```bash
>production       # inventory file for production servers
>staging          # inventory file for staging environment
>
>group_vars/
>   group1        # here we assign variables to particular
>   group2        # groups
>host_vars/
>   hostname1     # if systems need specific variables,
>   hostname2     # put them here
>
>library/         # if any custom modules (optional)
>module_utils/    # if any custom module_utils (optional)
>filter_plugins/  # if any custom filter plugins (optional)
>
>site.yml         # master playbook
>webservers.yml   # playbook for webserver tier
>dbservers.yml    # playbook for dbserver tier
>
>```

# Estructura recomanada (II)

>```bash
>roles/
>  common/          # this hierarchy represents a "role"
>    tasks/         
>      main.yml     # <- tasks file can include smaller files
>                   #    if warranted
>    handlers/      
>      main.yml     # <- handlers file
>    templates/     # <- files for use with the template
>                   #    resource
>      ntp.conf.j2  # <-- templates end in .j2
>    files/         #
>      bar.txt      # <- files for use with the copy resource
>      foo.sh       # <- script files for use with the script
>                   #    resource
>    vars/          #
>      main.yml     # <- variables associated with this role
>    defaults/      #
>      main.yml     # <- default lower priority variables
>                   #    for this role
>    meta/          #
>      main.yml     # <- role dependencies
>    library/       # roles can also include custom modules
>    module_utils/  # roles can also include custom 
>                   #     module_utils
>```

# Estructura dels inventaris (I)


## Que hi definim?

- **Hosts**: Utilitzant el _hostname_ o la seva _direcció IP_
- **Grups**: Agrupen hosts
- **Variables**: tant per un host o per un grup


# Estructura dels inventaris (II)

## Per a què serveixen els grups?

Els Grups ens permeten:

- Definir tasques (playbooks) específics per grups
- Definir variables específiques per grups
- Agrupar hosts segons "responsabilitats" o tipologia

Un Host sempre pertany com a mínim al grup "all".

Si un Host no té grup (a part del "all") s'afegeix al grup "ungrouped".

# Estructura dels inventaris (III)

##  Format clàssic (definició)

1. A no ser que la línia estigui enclausada amb ‘`[ ]`’, es tractarà com a host.
2. Si contenen únicament text entre '`[ ]`', representen un grup i tot el que segueix fins trobar un altre '`[ ]`' seran hosts d'aquest grup.
3. Si apart del text, conté el caràcter '`:`', representen atributs d'un grup, que seràn:
  - VARS:     Variables del grup, definides amb el format "`<variable>: <valor>`"
  - CHILDREN: Defineix quins grups són "fills" del grup actual
4. Es poden vincular variables a un host amb el format "`<variable>=<valor>`" separant les diferents variables amb espais.
  

# Estructura dels inventaris (III)

##  Format clàssic (exemple)


```
[web]
servidor_web  192.168.1.100

[documentacio]
docs_ansible  192.168.1.101  ansible_ssh_user=ansible src=/home/ansible/src

[web:children]
documentacio

[web:vars]
ansible_ssh_user: webmaster
```

# Estructura dels inventaris (IIII)

##  Format YAML (definició)

Utilitzarem el format d'arbre característic de YAML

```yaml
group:
    hosts:
        <host>:
            <nom_variable>: <valor_variable>
    children:
        group:
            ...
    vars:
        <nom_variable>: <valor_variable>
```



# Estructura dels inventaris (IIII)

##  Format YAML (exemple)

```yaml
web:
    hosts:
        servidor_web: 192.168.1.100
    children:
        documentacio:
            hosts:
                docs_ansible: 192.168.1.101
                    ansible_ssh_user: ansible
                    src: /home/ansible/src
    vars:
        ansible_ssh_user: webmaster
```

# Estructura dels playbooks (I)

## Format

Compleix el format Yaml, pel que tot el fitxer es representa en format d'arbre.

1. Estructura de bàsica
  - Definició de hosts afectats (`hosts`)
  - Definició de variables ANSIBLE
  - Definició de variables per defecte (`vars`)
  - Definició de tasques (`tasks`)
  - Definició de handlers (`handlers`)
  


# Estructura dels playbooks (II)

## Format

```yaml
---
- hosts: <grups o hostnames separats per ":">
  <nom de la variable ANSIBLE>: <valor de la variable ANSIBLE>
  vars:
    - <nom de la variable>: <valor de la variable>
  tasks:
    - name: <nom de la tasca>
      <nom del mòdul a utilitzar>:
        <variable del mòdul>: <valor de la variable>
      <paràmetre de tasca>: <valor del paràmetre>
  handlers:
    - name: <nom del handler>
      <nom del mòdul a utilitzar>:
        <variable del mòdul>: <valor de la variable>
      <paràmetre de tasca>: <valor del paràmetre>
```

# Estructura dels playbooks (III)

## Variables dels playbooks(I)

- Variables ANSIBLE
  - Aquelles que ja venen integrades a ANSIBLE
  - Són paràmetres per establir la connexió o realitzar la tasca
  - i.e.: 
    - ANSIBLE_SSH_USER: Usuari amb el que connectar-se
    - BECOME_USER: Usuari a convertir-se (sudo) quan s'estableixi la connexió
    - BECOME: Utilitzar SUDO
    - IMPORT_PLAYBOOK: Executa un playbook prèvi a l'actual

# Estructura dels playbooks (IV)

## Variables dels playbooks(II)

- Variables (VARS) dins la tasca
  - Les variables es poden accedir en format JINJA2 (`"{{ nom_variable }}"`)
  - Les variables es poden accedir mitjançant diccionaris (`nomgrup['nom_variable']`)

# Estructura dels playbooks (v)

## Variables dels playbooks(III)

- Paràmetres de tasca
  - Paràmetres que permeten lògica pre o post execució
  - i.e.:
    - WHEN: Funciona com un IF, només executa si es compleix la condició
    - NOTIFY: Llença un "signal" en acabar que ha de ser controlat per un HANDLER
    - REGISTER: Guarda el resultat de la tasca en una variable
    - IGNORE_ERRORS: Permet continuar el Playbook en cas d'error

# Estructura dels playbooks (III)

## Exemple

```
---
- hosts: webservers
  remote_user: root

  tasks:
  - name: write the apache config file
    template:
      src: /srv/httpd.j2
      dest: /etc/httpd.conf
    notify:
      - restart apache

  handlers:
    - name: restart apache
      service:
        name: apache
        state: restarted
```

# Definició de Rols (I)

- Permeten l'execució anidada de tasques
  - No cal que el playbook es trobi dins la carpeta de rols
  - Afegirem la variable ANSIBLE `roles`, que és una llista amb els rols afectats

```
---
- hosts: all
  roles:
    - el_meu_rol
```
  
- Cal definir-los en l'estructura de fitxers
  - Ho farem mantinguent l'estructura base, però ho colocarem sota dos directoris nous:
    - Roles: Directori base on es troben els rols
      - Nom del nostre rol: Directori on trobarem els fitxers relacionats amb el nostre rol
      


# Definició de Rols (II)

```
playbook.yml
roles/
  el_meu_rol/
    tasks/
    handlers/
    files/
    templates/
    vars/
    defaults/
    meta/
  un_altre_rol/
    tasks/
    handlers/
    files/
    templates/
    vars/
    defaults/
    meta/
```

# Documentació

- La documentació a ANSIBLE és molt important, ja que hi ha molts de mòduls i molts paràmetres
- Abans de fer qualsevol tasca, pensem que volem fer i busquem-ho
  - És molt probable que algú ja ho hagi fet
  - Hi poden haver mòduls que ens ajudin amb el que volem fer
  - I.e.: Actualitzar la configuració del "nostre" programa
    - Com que és nostre, ningú ha fet el mòdul
    - Sabem que la configuració és un fitxer de text
    - Sabem que només toquem una línia
    - El mòdul "lineinfile" permet actualitzar una línia mitjançant regex i variables
- Ansible és molt flexible, té molts de paràmetres que permeten fer moltes coses
  - Per aquest motiu és molt difícil recordar-los
  - Si busquem el que volem fer, és possible que ho trobem amb paràmetres
  - I.e.: Executar un playbook amb un usuari i una tasca amb sudo?
    - Afegim el paràmetre `become: yes` a la tasca

# Bibliografia

- Wikipedia: Idempotence - Wikipedia [Internet 2018] . Disponible al web: https://en.wikipedia.org/wiki/Idempotence
- Ansible: Ansible Documentation [Internet 2018]. Disponible al web: http://docs.ansible.com
  - [Ansible] Installation Guide -- Ansible Documentation. Disponible al web: http://docs.ansible.com/ansible/latest/installation_guide/intro_installation.html
  - [Ansible] Variables -- Ansible Documentation. Disponible al web: http://docs.ansible.com/ansible/latest/user_guide/playbooks_variables.html
  - [Ansible] Working with inventory -- Ansible Documentation. Disponible al web: http://docs.ansible.com/ansible/latest/user_guide/intro_inventory.html
  - [Ansible] Playbooks -- Ansible Documentation. Disponible al web: https://docs.ansible.com/ansible/2.3/playbooks.html
  - [Ansible] Working with playbooks -- Ansible Documentation. Disponible al web: http://docs.ansible.com/ansible/2.5/user_guide/playbooks_reuse_roles.html
  http://docs.ansible.com/ansible/latest/user_guide/playbooks.html
  - [Ansible] Roles -- Ansible Documentation. Disponible al web: http://docs.ansible.com/ansible/2.5/user_guide/playbooks_reuse_roles.html
  - [Ansible] All modules -- Ansible Documentation. Disponible al web: http://docs.ansible.com/ansible/latest/modules/list_of_all_modules.html
  - [Ansible] lineinfile - Manage lines in text files -- Ansible Documentation. Disponible al web: http://docs.ansible.com/ansible/latest/modules/lineinfile_module.html

# Moltes gràcies per escoltar

## Preguntes?

## Anem a provar-ho?